# Noise as target
A keras implementation of the [Unsupervised learning by predicting noise](https://arxiv.org/abs/1704.05310) paper  
  
Here's an analysis of the paper I rather enjoyed. [link](http://www.inference.vc/unsupervised-learning-by-predicting-noise-an-information-maximization-view-2/)  
  
This is an extraction and reformatting of the implementation my team and I made as part of the "[Mozgalo](https://www.estudent.hr/category/natjecanja/mozgalo/)" competition.  
  
I have the [celebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset on hand, and it's a decently big and publicly available dataset unlike mozgalo so this demo uses it.  

In [3]:
%matplotlib inline

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D

from scipy.optimize import linear_sum_assignment

from tqdm import tqdm

import matplotlib.pyplot as plt

import time
from itertools import cycle, islice

Using TensorFlow backend.


### config

In [4]:
image_shape = (224,224)
batch_size = 128
n_features = 1000

## Building the model
This is a [darknet](https://pjreddie.com/darknet/imagenet/) model.  
You can use whatever you wish. One of the things I find so nice about this method is that the models are interchangeable with standard classification models

In [5]:
def convolutional_block(inp, filters):
    x = Conv2D(filters=filters, kernel_size=3, strides=(1,1), padding='same', use_bias=False)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=2)(x)
    return x

c = 1  # we train on b&w images, as the paper suggests, to make the training objective harder 


input_img = Input(shape=(*image_shape, c))

x = convolutional_block(input_img, 16)

x = convolutional_block(x, 32)

x = convolutional_block(x, 64)

x = convolutional_block(x, 128)

x = convolutional_block(x, 256)

x = convolutional_block(x, 512)

x = convolutional_block(x, 1024)

x = Conv2D(filters=n_features, kernel_size=1, strides=(1,1), padding='same', use_bias=False)(x)
x = LeakyReLU()(x)
x = GlobalAveragePooling2D()(x)

model = Model(inputs=input_img, outputs=x)
model.compile(loss='mean_squared_error',
              optimizer='adam')

# print(model.summary())

### or if you prefer, load a trained model

In [ ]:
model = load_model('../models/something')

## Helper functions for training

In [6]:
def rand_unit_sphere(npoints, ndim=1000):
    '''
    Generates "npoints" number of vectors of size "ndim"
    such that each vectors is a point on an "ndim" dimensional sphere
    that is, so that each vector is of distance 1 from the center
    
    npoints -- number of feature vectors to generate
    ndim -- how many features per vector
    
    returns -- np array of shape (npoints, ndim), dtype=float64
    '''
    vec = np.random.randn(npoints, ndim)
    vec = np.divide(vec, np.expand_dims(np.linalg.norm(vec, axis=1), axis=1))
    return vec

In [7]:
def noises_from_keys(noises, keys):
    '''
    returns the assigned noise vectors for a batch of images
    '''
    keys = list(keys)
    n = np.empty(shape=(len(keys), n_features), dtype='float64')
    for i in range(len(keys)):
        n[i] = noises[keys[i]]
        
    return n

In [8]:
def shuffle_assigned_noises(noises):
    '''
    shuffles all of the noises assigned to images
    done evety N epoch to avoid plateaus
    '''
    new_noises = {}
    keys = noises.keys()
    values = list(noises.values())
    np.random.shuffle(values)
    
    for k, v in zip(keys, values):
        new_noises[k] = v
        
    return new_noises

## Loading the data!

In [9]:
def datagen_wrapper(datagen, noises):
    filename_iter = islice(cycle(datagen.filenames), datagen.batch_size)
    
    while True:
        ns = noises_from_keys(noises, filename_iter.__next__())
        yield datagen.__next__(), ns
    

In [ ]:
datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

datagen = datagen.flow_from_directory(
    '../data/',
    target_size=image_shape,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False,
    color_mode='grayscale'
    )

n_images = datagen.n

Found 202599 images belonging to 1 classes.


In [53]:
unit_sphere_noises = rand_unit_sphere(n_images, n_features)

In [54]:
assigned_noises = {}

for fn, ft in zip(datagen.filenames, unit_sphere_noises):
    assigned_noises[fn] = ft

In [2]:
gen = datagen_wrapper(datagen, assigned_noises)

NameError: name 'datagen_wrapper' is not defined

In [60]:
x, y = gen.__next__()
print(x.shape)
print(y.shape)
#gen_wrapper(gen, filenames, noises)

['i', 'm', 'g', '_', 'a', 'l', 'i', 'g', 'n', '_', 'c', 'e', 'l', 'e', 'b', 'a', '/', '1', '1', '2', '4', '2', '3', '.', 'j', 'p', 'g']


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
list(cycle(datagen.filenames))[:20]

In [ ]:
epochs_per_shuffle=3

# base name of model save file - based on training start time
save_dir = '../models/'
folder_name = time.strftime("%Y-%m-%d-%H-%M/", time.gmtime())
save_dir = save_dir + folder_name
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

epoch = 1
while True:
    print('Running epoch:', epoch)
    for b in range(3):
        for i in tqdm(n_images / batch_size))):
            # get image batch
            images, keys = batch_generator.__next__()
            # get noise mapped to each image in batch
            n = noises_from_keys(keys)

            # do forward pass on batch - features
            losses = np.empty(shape=(batch_size, batch_size), dtype='float64')
            features = model.predict_on_batch(images)
            
            # calculate l2 loss between all...
            # noises randomly assigned and features generated by the network
            for b in range(batch_size):
                fts = np.repeat(np.expand_dims(features[b], axis = 0), batch_size, axis=0)
                l2 = np.linalg.norm(fts - n, axis=1)
                losses[b] = l2
            
            # rearrange noises such that the total loss is minimal (hungarian algorithm)
            row_ind, col_ind = linear_sum_assignment(losses)
            for r, c in zip(row_ind, col_ind):
                noises[keys[r]] = n[c]
                
            n = noises_from_keys(keys)
            
            # do a backprop pass with the newly arranged features
            model.train_on_batch(images, n)
            
        # every epoch save model
        model.save(save_dir  + str(epoch) + '.model')
        epoch += 1
            
    # every cca 3 epochs shuffle noises
    noises = shuffle_noises(noises)